# Microstructure Analysis of EURUSD via Hawkes Processes
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artbert/fx-hawkes-microstructure/blob/main/notebook.ipynb)

# Project Overview
Analiza dynamiki mikrostruktury rynku FX (EURUSD) przy użyciu samowzbudzających się procesów punktowych. Projekt bada przejście od prostej statystyki opisowej, przez modelowanie 1D (intensywność klastrowania), aż po interakcje Bid-Ask w modelu 2D.

**Key Tech Stack:** Python, Pandas, SciPy, Hawkes Processes (MLE).

In [ ]:
# ==============================================================================
# COLAB SETUP (Uruchom tylko jeśli korzystasz z Google Colab)
# ==============================================================================
import os
import sys

if 'google.colab' in str(get_ipython()):
    print("Running on Colab. Cloning repository and installing dependencies...")

    # Klonowanie repozytorium, jeśli folder 'src' nie istnieje
    if not os.path.exists('src'):
        !git clone https://github.com/artbert/fx-hawkes-microstructure.git .

    # Instalacja bibliotek z requirements.txt
    !pip install -r requirements.txt

    # Dodanie bieżącego katalogu do ścieżki, aby Python widział folder 'src'
    sys.path.append(os.getcwd())
    print("Setup complete.")

# 01. Data Preparation & Validation

Celem etapu jest transformacja surowych szeregów tickowych do postaci procesów punktowych. Pipeline obejmuje unifikację kwotowań, filtrację okresów niskiej płynności (rollover) oraz inżynierię cech mikrostrukturalnych.

## 1.1 Environment Setup & Dependency Management
Inicjalizacja środowiska i import modułów odpowiedzialnych za procesy ETL oraz kalkulacje statystyczne.

In [ ]:
import pandas as pd
import numpy as np
import datetime
from src.utils import load_and_prepare, add_tick_features

## 1.2 Data Ingestion & Integrity Check
Ładowanie danych z uwzględnieniem filtracji okresu rollover (23:00 – 01:15), w którym dynamika rynku jest zaburzona przez niską płynność.

In [ ]:
file_path = "data/samples/EURUSD_2025_10_01.csv"
df = load_and_prepare(file_path, drop_rollover=True, rollover_start="23:00", rollover_end="01:15")

# Walidacja integralności i brakujących danych
assert 'timestamp' in df.columns
print(df[['timestamp','bid_raw','ask_raw']].head())
print(f"Rows: {len(df)}; NaN bid_raw: {df['bid_raw'].isna().sum()}; NaN ask_raw: {df['ask_raw'].isna().sum()}")

## 1.3 Feature Synthesis
Generowanie kluczowych zmiennych: $\Delta t$ (czas między tickami), $\Delta price$ oraz dynamika spreadu.

In [ ]:
df = add_tick_features(df)
df_sample = df.head(200)
df_sample[['timestamp','bid','ask','bid_raw','ask_raw','delta_bid','delta_ask','delta_mid_q','spread']].head(20)

# Eksport przetworzonego zbioru do formatu Parquet (efektywność I/O)
df.to_parquet("data/cleaned_EURUSD_2025_10_01.parquet", index=False)

# 02. Exploratory Data Analysis (EDA)

Przed modelowaniem Hawkesa przeprowadzono diagnostykę rozkładów w celu identyfikacji cech charakterystycznych dla mikrostruktury HFT: klastrowania i grubych ogonów.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from src.utils import plot_distribution, basic_distribution

sns.set(style="whitegrid")
df = pd.read_parquet("data/cleaned_EURUSD_2025_10_01.parquet")

## 2.1 Analiza Rozkładów i Autokorelacji
Wizualizacja dynamiki zmian cen oraz czasu między aktualizacjami arkusza zleceń.

In [ ]:
# Rozkłady zmian cen i spreadu
plot_distribution(df['delta_bid'], title="Delta Bid Distribution")
plot_distribution(df['delta_ask'], title="Delta Ask Distribution")
plot_distribution(df['spread'], title="Spread Distribution")
plot_distribution(df['delta_mid_q'], title="Quote-based Mid Change Distribution")

# Statystyki opisowe i diagnostyka autokorelacji
features = ['delta_bid','delta_ask','delta_mid_q','delta_t_bid','delta_t_ask','spread']
summary = {f: basic_distribution(df[f]) for f in features}
print(json.dumps(summary, indent=2))

def autocorr(series, lag):
    x = series.dropna()
    return x.autocorr(lag=lag)

for lag in [1,5,10]:
    print(f"Lag {lag}: delta_bid={autocorr(df['delta_bid'], lag):.4f}, delta_ask={autocorr(df['delta_ask'], lag):.4f}")

> **Kluczowe obserwacje:**
> 1. **Grube ogony:** Rozkłady wykazują wysoką kurtozę – rynek reaguje gwałtownymi skokami aktywności.
> 2. **Asynchroniczność:** Zmienność Bid/Ask przewyższa zmienność Mid-quote, co czyni Mid-price niewystarczającym przybliżeniem dla modeli HFT.
> 3. **Klastrowanie:** Ticki nie tworzą procesu Poissona; obserwujemy serie aktywności (bursts) typowe dla handlu algorytmicznego.
>
>

# 03. Hawkes Modeling (1D)
Modelowanie intensywności zdarzeń $\lambda(t)$ jako sumy bazowej aktywności ($\mu$) oraz efektu samowzbudzania (jądro wykładnicze $\alpha e^{-\beta \Delta t}$).

## 3.1 Model Stały (Baseline)
Wstępna estymacja parametrów przy założeniu stałej intensywności bazowej.

In [ ]:
import numpy as np
import pandas as pd
from src.utils import prepare_times, fit_hawkes_1d

df = pd.read_parquet("data/cleaned_EURUSD_2025_10_01.parquet")
bid_times = df.loc[df['bid_raw'].notna(), 'timestamp']
ask_times = df.loc[df['ask_raw'].notna(), 'timestamp']

# Przygotowanie szeregów czasowych
T_bid = prepare_times(bid_times)
T_ask = prepare_times(ask_times)
T_all = prepare_times(df['timestamp'])
itv_bid = [0.0, T_bid[-1]]
itv_ask = [0.0, T_ask[-1]]
itv_all = [0.0, T_all[-1]]

params_all_const = fit_hawkes_1d(T_all, itv_all, baseline='const')
print(f"Full stream params: {params_all_const}")

params_bid_const = fit_hawkes_1d(T_bid, itv_bid, baseline='const')
params_ask_const = fit_hawkes_1d(T_ask, itv_ask, baseline='const')
print(f"Bids stream params: {params_bid_const}")
print(f"Asks stream params: {params_ask_const}")

Estymacja wskazuje na ekstremalnie wysoką endogeniczność (Branching Ratio $\alpha \approx 0.89$). Dla strumienia bid i ask oddzielnie jest to wartość niewiele mniejsza. System wydaje się bliski stanu krytycznego, co sugeruje dominację reakcji algorytmicznych.

## 3.2 Dekompozycja Sezonowości (`pconst`)
Zastosowanie schodkowej funkcji intensywności bazowej w celu odizolowania cyklu dobowego FX od realnego samowzbudzania.

In [ ]:
from src.utils import plot_num_basis_vs_alpha

# Analiza wpływu granularności baseline na parametr alpha
different_num_basis_stats = {}
alpha_bids = []
alpha_asks = []
num_basis_values = [6, 12, 18, 24]
for num_basis in num_basis_values:
    params_bid_pconst = fit_hawkes_1d(T_bid, itv_bid, baseline='pconst', num_basis=num_basis)
    params_ask_pconst = fit_hawkes_1d(T_ask, itv_ask, baseline='pconst', num_basis=num_basis)
    different_num_basis_stats[num_basis] = params_bid_pconst
    alpha_bids.append(params_bid_pconst['alpha'])
    alpha_asks.append(params_ask_pconst['alpha'])

# Prepare line plot for num_basis_values vs alpha_bids and alpha_asks
plot_num_basis_vs_alpha(num_basis_values, alpha_bids, alpha_asks)
print(different_num_basis_stats)

Wprowadzenie zmiennej bazy $\mu(t)$ (podział na segmenty czasowe) pozwoliło oddzielić cykl dobowy FX od mikro-kaskad.

| Metryka | Model Stały | Model Sezonowy (`pconst`) |
| --- | --- | --- |
| **Endogeniczność ($\alpha$)** | ~84% | **~50-60%** |
| **Pamięć rynku ($1/\beta$)** | ~3.0s | **~0.8s** |
| **Dopasowanie (AIC)** | Bazowe | **Znacznie lepsze** |

**Wniosek:** Ignorowanie sezonowości intraday prowadzi do przeszacowania siły klastrowania. Realna pamięć mikrostruktury EURUSD wynosi poniżej 1 sekundy.

# 04. Multivariate Hawkes (2D: Bid-Ask Interaction)

Analiza interakcji między ofertami kupna i sprzedaży. Model bada, w jakim stopniu zmiana ceny po jednej stronie arkusza wymusza reakcję po drugiej.

## 4.1 Estymacja MLE i Stabilność Numeryczna
Implementacja pełnego modelu 2D z wykorzystaniem optymalizacji L-BFGS-B.

In [ ]:
from scipy.optimize import minimize
from src.utils import hawkes_2d_loglik

df = pd.read_parquet("data/cleaned_EURUSD_2025_10_01.parquet")
df = df[['timestamp', 'bid_raw', 'ask_raw']]
# wspólny punkt odniesienia
t0 = df['timestamp'].iloc[0]

T_bid = prepare_times(df.loc[df['bid_raw'].notna(), 'timestamp'], t0)
T_ask = prepare_times(df.loc[df['ask_raw'].notna(), 'timestamp'], t0)

T_end = (df['timestamp'].iloc[-1] - t0).total_seconds()

init_params = np.array([
    0.15,   # mu_b
    0.15,   # mu_a
    0.4,    # a_bb
    0.1,    # a_ba
    0.1,    # a_ab
    0.4,    # a_aa
    1.0     # beta
])

bounds = [(1e-6, None)] * 7

result = minimize(
    hawkes_2d_loglik,
    init_params,
    args=(T_bid, T_ask, T_end, False),
    bounds=bounds,
    method="L-BFGS-B"
)

params_hat = result.x

mu_b, mu_a, a_bb, a_ba, a_ab, a_aa, beta = params_hat

A = np.array([[a_bb, a_ba],
              [a_ab, a_aa]])

eigvals = np.linalg.eigvals(A)
spectral_radius = max(np.abs(eigvals))

print("Macierz A:")
print(A)
print("Spektralny radius:", spectral_radius)
print("mu_b:", mu_b)
print("mu_a:", mu_a)
print("beta: ",beta)

### Hawkes 2D z ograniczonym β

In [ ]:
# Inicjalizacja optymalizacji z restrykcją na parametr beta (0.2 - 3.0)
# Zapobiega to numerycznej degeneracji modelu przy danych o rozdzielczości 1ms
init_params = np.array([
    0.2,   # mu_b
    0.2,   # mu_a
    0.3,   # a_bb
    0.1,   # a_ba
    0.1,   # a_ab
    0.3,   # a_aa
    1.0    # beta
])

bounds = [
    (1e-6, None),  # mu_b
    (1e-6, None),  # mu_a
    (1e-6, None),  # a_bb
    (1e-6, None),  # a_ba
    (1e-6, None),  # a_ab
    (1e-6, None),  # a_aa
    (0.2, 3.0)     # <-- beta ograniczone
]

result = minimize(
    hawkes_2d_loglik,
    init_params,
    args=(T_bid, T_ask, T_end, True),
    bounds=bounds,
    method="L-BFGS-B"
)

params_hat = result.x

mu_b, mu_a, a_bb, a_ba, a_ab, a_aa, beta = params_hat

A = np.array([[a_bb, a_ba],
              [a_ab, a_aa]])

eigvals = np.linalg.eigvals(A)
spectral_radius = max(np.abs(eigvals))

print("Macierz A:")
print(A)
print("Spektralny radius:", spectral_radius)
print("mu_b:", mu_b)
print("mu_a:", mu_a)
print("beta: ",beta)

### Wyzwania numeryczne i techniczne

Pierwotna próba estymacji MLE wykazała degenerację modelu ($\beta \to \infty$).

* **Przyczyna:** Wysoka agregacja danych (50% rekordów ma identyczny timestamp dla Bid i Ask).
* **Rozwiązanie:** Nałożenie restrykcji na parametr $\beta$ (0.2 - 3.0), aby dopasować model do fizycznych możliwości infrastruktury rynkowej.

### Wyniki modelu 2D

Interpretacja macierzy wzbudzeń $A$ oraz promienia spektralnego $ρ(A)$.

* **Dominacja Cross-Excitation:** Samowzbudzanie wewnątrz jednej strony jest niskie (\~0.07). Kluczowym driverem jest interakcja krzyżowa (\~0.65).
* **Dynamika:** Rynek to „dialog” – ruch jednej strony wymusza niemal natychmiastową (ok. 0.3s) korektę drugiej strony w celu zbalansowania spreadu.

# 05. Summary & Data Quality Diagnostics
Końcowa weryfikacja jakości danych w celu zrozumienia ograniczeń modelu 2D.

In [ ]:
# Analiza duplikacji timestampów i powtarzalności kwotowań
df = pd.read_parquet("data/cleaned_EURUSD_2025_10_01.parquet")
df = df[['timestamp', 'bid_raw', 'ask_raw']]
original_len = len(df)
print(f"Simultaneous Bid/Ask updates: {(df['bid_raw'].notna() & df['ask_raw'].notna()).sum()}")

df['ask_diff'] = df['ask_raw'].diff() * 100000
df['bid_diff'] = df['bid_raw'].diff() * 100000
df = df.dropna()
df['ask_diff'] = df['ask_diff'].astype(int)
df['bid_diff'] = df['bid_diff'].astype(int)
print(f"Number of records where the bid price value was repeated: {df['bid_diff'].value_counts()[0]}")
print(f"Number of records where the ask price value was repeated: {df['ask_diff'].value_counts()[0]}")
print(f"Percentage of records where the bid was repeated: {df['bid_diff'].value_counts()[0]/original_len:.0%}")
print(f"Percentage of records where the ask was repeated: {df['ask_diff'].value_counts()[0]/original_len:.0%}")

### Finalne wnioski:

1. **Stabilność:** Rynek EURUSD jest systemem stabilnym ($ρ(A) < 1$), wysoce reaktywnym i efektywnym.
2. **Struktura:** Dynamika opiera się na trzech warstwach: makro-sezonowości, błyskawicznych reakcji krzyżowych (cross-excitation) oraz braku długiej pamięci trendu.
3. **Limit Techniczny:** Rozdzielczość 1ms jest "wąskim gardłem" dla modelu 2D – około 50% zdarzeń dzieje się symultanicznie, co wymaga danych mikrosekundowych do pełnej separacji procesów.